In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import train_test_split
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic
from surprise import accuracy
from surprise import dump


In [2]:
df = pd.read_csv('data/df4.csv')

df.head()

In [3]:
df.head(10)

,customer_id,article_id,rfm,count,user_id,item_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001.0,5.472143,1,2,40179
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023.0,3.484524,1,2,10520
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004.0,1.993809,1,7,6387
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003.0,1.993809,1,7,46304
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004.0,1.993809,1,7,46305
5,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001.0,1.993809,1,7,46302
6,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001.0,2.490714,1,7,6386
7,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012.0,3.484524,1,198,47416
8,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011.0,4.975238,1,198,5944
9,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003.0,4.975238,1,198,22540


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   customer_id  object 
 1   article_id   float64
 2   rfm          float64
 3   count        int64  
 4   user_id      int64  
 5   item_id      int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ GB


In [3]:
rating = df.copy()
rating.drop(['customer_id','article_id', 'count'], axis=1)

,rfm,user_id,item_id
0,5.472143,2,40179
1,3.484524,2,10520
2,1.993809,7,6387
3,1.993809,7,46304
4,1.993809,7,46305
...,...,...,...
31788319,9.503095,1371691,104961
31788320,10.000000,1371691,100629
31788321,9.503095,1371721,104053
31788322,5.527857,1371747,88521


In [4]:
rating = rating[['user_id', 'item_id', 'rfm']]
rating = rating.rename(columns={'user_id':'user','item_id':'item', 'rfm':'rating'})

In [1]:
rating.head()

NameError: name 'rating' is not defined

In [5]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(rating[['user', 'item', 'rating']], reader)

# # We can now use this dataset as we please, e.g. calling cross_validate
# cross_validate(NormalPredictor(), data, cv=2)

In [6]:
train, test = train_test_split(data, test_size=0.25, random_state=42)

In [7]:
algo = NMF()
algo.fit(train)

In [8]:
prediction = algo.test(test)

In [9]:
dump.dump('./dump_filenmf', prediction, algo)

In [ ]:
predictions, algo = dump.load('./dump_filenmf')

In [10]:
accuracy.rmse(prediction)

RMSE: 1.9948


1.9948421376495724

In [11]:
accuracy.mae(prediction)

MAE:  1.5481


1.5480708523060764

In [41]:
print ('prediction type:', type(prediction),'size: ', len(prediction))
print()
print('prediction 결과값 5개 보기')
print(prediction[:5])

prediction type: <class 'list'> size:  7947081

prediction 결과값 5개 보기
[Prediction(uid=23501, iid=9934, r_ui=2.993809457314158, est=5, details={'was_impossible': False}), Prediction(uid=1171330, iid=44595, r_ui=2.5030952713429206, est=3.861822589092995, details={'was_impossible': False}), Prediction(uid=1193048, iid=56999, r_ui=5.496904728657079, est=5, details={'was_impossible': False}), Prediction(uid=326791, iid=10648, r_ui=6.006190542685841, est=5, details={'was_impossible': False}), Prediction(uid=1360077, iid=67237, r_ui=6.496904728657079, est=5, details={'was_impossible': False})]


In [11]:
prediction = algo.test(test)

AttributeError: 'SVD' object has no attribute 'train'

In [12]:
df = pd.DataFrame(prediction, columns=['uid', 'iid', 'rui', 'est', 'details'])

In [13]:
df.head()

,uid,iid,rui,est,details
0,23501,9934,2.993809,3.677826,{'was_impossible': False}
1,1171330,44595,2.503095,3.448775,{'was_impossible': False}
2,1193048,56999,5.496905,5.000000,{'was_impossible': False}
3,326791,10648,6.006191,4.528192,{'was_impossible': False}
4,1360077,67237,6.496905,5.000000,{'was_impossible': False}


In [14]:
np.mean(np.abs((df['rui'] - df['est']) / df['rui'])) * 100 

25.44733660749468